## Įterpimai

Ankstesniame pavyzdyje dirbome su aukštos dimensijos žodžių maišo vektoriais, kurių ilgis yra `vocab_size`, ir aiškiai konvertavome žemos dimensijos pozicinius reprezentacijos vektorius į retą vieno karšto (one-hot) reprezentaciją. Ši vieno karšto reprezentacija nėra efektyvi atminties požiūriu. Be to, kiekvienas žodis yra traktuojamas nepriklausomai nuo kitų, todėl vieno karšto kodavimo vektoriai neišreiškia semantinių panašumų tarp žodžių.

Šiame skyriuje toliau tyrinėsime **News AG** duomenų rinkinį. Pradėkime įkeldami duomenis ir pasinaudodami ankstesnio skyriaus apibrėžimais.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Kas yra įterpimas?

**Įterpimo** idėja yra atvaizduoti žodžius naudojant mažesnės dimensijos tankius vektorius, kurie atspindi žodžio semantinę prasmę. Vėliau aptarsime, kaip sukurti prasmingus žodžių įterpimus, tačiau kol kas galvokime apie įterpimus kaip apie būdą sumažinti žodžio vektoriaus dimensiją.

Taigi, įterpimo sluoksnis priima žodį kaip įvestį ir pateikia išvesties vektorių su nurodytu `embedding_size`. Tam tikra prasme jis yra labai panašus į `Dense` sluoksnį, tačiau vietoj to, kad naudotų vieno karšto kodavimo (one-hot encoding) vektorių kaip įvestį, jis gali priimti žodžio numerį.

Naudodami įterpimo sluoksnį kaip pirmąjį mūsų tinklo sluoksnį, galime pereiti nuo žodžių maišo (bag-of-words) prie **įterpimo maišo** (embedding bag) modelio. Šiame modelyje pirmiausia kiekvieną tekstą sudarantį žodį paverčiame atitinkamu įterpimu, o tada apskaičiuojame tam tikrą agregavimo funkciją visiems tiems įterpimams, pavyzdžiui, `sum`, `average` arba `max`.

![Vaizdas, rodantis įterpimo klasifikatorių penkiems sekos žodžiams.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Mūsų klasifikatoriaus neuroninis tinklas susideda iš šių sluoksnių:

* `TextVectorization` sluoksnis, kuris priima eilutę kaip įvestį ir pateikia skaičių žetonų (tokenų) tensorių. Mes nurodysime tam tikrą pagrįstą žodyno dydį `vocab_size` ir ignoruosime rečiau naudojamus žodžius. Įvesties forma bus 1, o išvesties forma bus $n$, nes gausime $n$ žetonų kaip rezultatą, kurių kiekvienas turės skaičius nuo 0 iki `vocab_size`.
* `Embedding` sluoksnis, kuris priima $n$ skaičių ir sumažina kiekvieną skaičių iki tankaus vektoriaus su nurodytu ilgiu (mūsų pavyzdyje – 100). Taigi, įvesties tensorius su forma $n$ bus transformuotas į $n\times 100$ tensorių.
* Agregavimo sluoksnis, kuris apskaičiuoja šio tensoriaus vidurkį pagal pirmąją ašį, t. y. jis apskaičiuos visų $n$ įvesties tensorių, atitinkančių skirtingus žodžius, vidurkį. Šiam sluoksniui įgyvendinti naudosime `Lambda` sluoksnį ir perduosime jam funkciją, skirtą vidurkiui apskaičiuoti. Išvestis turės 100 formą ir bus visos įvesties sekos skaitmeninis atvaizdavimas.
* Galutinis `Dense` linijinis klasifikatorius.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Santraukoje, stulpelyje **output shape**, pirmoji tensoriaus dimensija `None` atitinka mini partijos dydį, o antroji - žetonų sekos ilgį. Visos žetonų sekos mini partijoje turi skirtingus ilgius. Apie tai, kaip su tuo susidoroti, aptarsime kitame skyriuje.

Dabar treniruokime tinklą:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Pastaba**: mes kuriame vektorizatorių remdamiesi duomenų pogrupiu. Tai daroma siekiant pagreitinti procesą, ir tai gali lemti situaciją, kai ne visi mūsų teksto žetonai yra žodyne. Tokiu atveju tie žetonai būtų ignoruojami, o tai gali šiek tiek sumažinti tikslumą. Tačiau realiame gyvenime teksto pogrupis dažnai suteikia gerą žodyno įvertinimą.


### Darbas su kintamais sekų dydžiais

Pažvelkime, kaip vyksta mokymas naudojant mini partijas. Aukščiau pateiktame pavyzdyje įvesties tensorius turi dimensiją 1, o mes naudojame 128 dydžio mini partijas, todėl tikrasis tensoriaus dydis yra $128 \times 1$. Tačiau kiekviename sakinyje esančių žodžių skaičius skiriasi. Jei pritaikysime `TextVectorization` sluoksnį vienai įvesčiai, grąžinamų žodžių skaičius bus skirtingas, priklausomai nuo to, kaip tekstas yra suskaidytas į žodžius:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Tačiau, kai vektorizatorių taikome kelioms sekoms, jis turi sukurti stačiakampio formos tensorių, todėl neužpildytus elementus užpildo PAD žetonu (kuris mūsų atveju yra nulis):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Čia matome įterpimus:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Pastaba**: Siekiant sumažinti užpildymo kiekį, kai kuriais atvejais prasminga surūšiuoti visas sekas duomenų rinkinyje didėjančio ilgio tvarka (arba, tiksliau, pagal žetonų skaičių). Tai užtikrins, kad kiekvienas mažasis paketas turės panašaus ilgio sekas.


## Semantiniai įterpiniai: Word2Vec

Mūsų ankstesniame pavyzdyje įterpimo sluoksnis išmoko susieti žodžius su vektorinėmis reprezentacijomis, tačiau šios reprezentacijos neturėjo semantinės prasmės. Būtų naudinga išmokti vektorinę reprezentaciją, kurioje panašūs žodžiai ar sinonimai atitiktų vektorius, esančius arti vienas kito pagal tam tikrą vektorinį atstumą (pavyzdžiui, euklidinį atstumą).

Tam reikia iš anksto apmokyti mūsų įterpimo modelį naudojant didelę tekstų kolekciją ir tokią techniką kaip [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ši technika remiasi dviem pagrindinėmis architektūromis, kurios naudojamos žodžių paskirstytai reprezentacijai kurti:

 - **Nuolatinis žodžių maišas** (CBoW), kai modelis yra mokomas numatyti žodį pagal aplinkinį kontekstą. Turint ngramą $(W_{-2},W_{-1},W_0,W_1,W_2)$, modelio tikslas yra numatyti $W_0$ pagal $(W_{-2},W_{-1},W_1,W_2)$.
 - **Nuolatinis skip-gramas** yra priešingas CBoW. Modelis naudoja aplinkinį kontekstinių žodžių langą, kad numatytų dabartinį žodį.

CBoW yra greitesnis, o skip-gramas, nors ir lėtesnis, geriau reprezentuoja retai pasitaikančius žodžius.

![Vaizdas, rodantis tiek CBoW, tiek Skip-Gram algoritmus, skirtus žodžiams paversti vektoriais.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Norėdami eksperimentuoti su Word2Vec įterpimu, iš anksto apmokytu naudojant Google News duomenų rinkinį, galime naudoti **gensim** biblioteką. Žemiau pateikiame žodžius, kurie yra labiausiai panašūs į 'neural'.

> **Note:** Kai pirmą kartą kuriate žodžių vektorius, jų atsisiuntimas gali užtrukti!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Taip pat galime išgauti vektoriaus įterpimą iš žodžio, kuris bus naudojamas klasifikavimo modelio mokymui. Įterpimas turi 300 komponentų, tačiau čia aiškumo dėlei parodome tik pirmuosius 20 vektoriaus komponentų:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Puikus dalykas apie semantinius įterpimus yra tai, kad galite manipuliuoti vektoriaus kodavimu pagal semantiką. Pavyzdžiui, galime paprašyti surasti žodį, kurio vektorinė reprezentacija būtų kuo artimesnė žodžiams *karalius* ir *moteris*, ir kuo toliau nuo žodžio *vyras*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Pavyzdys aukščiau naudoja tam tikrą vidinę GenSym magiją, tačiau pagrindinė logika iš tikrųjų yra gana paprasta. Įdomus dalykas apie įterpimus yra tai, kad galite atlikti įprastas vektorių operacijas su įterpimo vektoriais, ir tai atspindėtų operacijas su žodžių **reikšmėmis**. Pavyzdys aukščiau gali būti išreikštas vektorių operacijų terminais: mes apskaičiuojame vektorių, atitinkantį **KARALIUS-VYRAS+MOTERIS** (operacijos `+` ir `-` atliekamos su atitinkamų žodžių vektorinėmis reprezentacijomis), o tada randame artimiausią žodį žodyne tam vektoriui:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **PASTABA**: Turėjome pridėti mažus koeficientus prie *vyro* ir *moters* vektorių – pabandykite juos pašalinti ir pažiūrėkite, kas nutiks.

Norėdami rasti artimiausią vektorių, naudojame TensorFlow įrankius, kad apskaičiuotume atstumų vektorių tarp mūsų vektoriaus ir visų žodyno vektorių, o tada naudojame `argmin`, kad rastume mažiausio žodžio indeksą.


Nors Word2Vec atrodo puikus būdas išreikšti žodžių semantiką, jis turi daug trūkumų, įskaitant šiuos:

* Tiek CBoW, tiek skip-gram modeliai yra **prognozuojamieji įterpiniai**, ir jie atsižvelgia tik į vietinį kontekstą. Word2Vec nepasinaudoja globaliu kontekstu.
* Word2Vec neatsižvelgia į žodžių **morfologiją**, t. y. į tai, kad žodžio reikšmė gali priklausyti nuo skirtingų žodžio dalių, pavyzdžiui, šaknies.

**FastText** bando įveikti antrąjį apribojimą ir remiasi Word2Vec, mokydamas vektorių reprezentacijas kiekvienam žodžiui ir simbolių n-gramoms, randamoms kiekviename žodyje. Reprezentacijų reikšmės tada vidurkinamos į vieną vektorių kiekviename mokymo žingsnyje. Nors tai prideda daug papildomų skaičiavimų išankstiniam mokymui, tai leidžia žodžių įterpiniams užkoduoti subžodžių informaciją.

Kitas metodas, **GloVe**, naudoja kitokį požiūrį į žodžių įterpinius, pagrįstą žodžių-konteksto matricos faktorizacija. Pirmiausia jis sukuria didelę matricą, kuri skaičiuoja žodžių pasikartojimų skirtinguose kontekstuose skaičių, o tada bando šią matricą pavaizduoti mažesnėje dimensijoje taip, kad būtų sumažintas rekonstrukcijos nuostolis.

Gensim biblioteka palaiko šiuos žodžių įterpinius, ir jūs galite eksperimentuoti su jais, pakeisdami aukščiau pateiktą modelio įkėlimo kodą.


## Naudojant iš anksto apmokytus įterpimus Keras bibliotekoje

Galime pakeisti aukščiau pateiktą pavyzdį, kad užpildytume įterpimo sluoksnio matricą semantiniais įterpimais, tokiais kaip Word2Vec. Iš anksto apmokyto įterpimo ir teksto korpuso žodynai greičiausiai nesutaps, todėl turime pasirinkti vieną. Čia nagrinėjame dvi galimas parinktis: naudoti žodyną iš tokenizerio arba naudoti žodyną iš Word2Vec įterpimų.

### Naudojant tokenizerio žodyną

Naudojant tokenizerio žodyną, kai kurie žodžiai iš žodyno turės atitinkamus Word2Vec įterpimus, o kai kurių trūks. Atsižvelgiant į tai, kad mūsų žodyno dydis yra `vocab_size`, o Word2Vec įterpimo vektoriaus ilgis yra `embed_size`, įterpimo sluoksnis bus atvaizduotas svorio matrica, kurios forma yra `vocab_size`$\times$`embed_size`. Šią matricą užpildysime eidami per žodyną:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Žodžiams, kurie nėra Word2Vec žodyno dalis, galime palikti jų reikšmes kaip nulius arba sugeneruoti atsitiktinį vektorių.

Dabar galime apibrėžti įterpimo sluoksnį su iš anksto paruoštais svoriais:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Pastaba**: Atkreipkite dėmesį, kad nustatėme `trainable=False` kurdami `Embedding`, tai reiškia, kad mes nepertreniruojame Embedding sluoksnio. Tai gali šiek tiek sumažinti tikslumą, tačiau paspartina mokymą.

### Naudojant įterpimo žodyną

Viena problema su ankstesniu metodu yra ta, kad TextVectorization ir Embedding naudoja skirtingus žodynus. Norėdami išspręsti šią problemą, galime naudoti vieną iš šių sprendimų:
* Iš naujo pertreniruoti Word2Vec modelį pagal mūsų žodyną.
* Įkelti mūsų duomenų rinkinį naudojant žodyną iš iš anksto apmokyto Word2Vec modelio. Žodynai, naudojami duomenų rinkiniui įkelti, gali būti nurodyti įkėlimo metu.

Antrasis metodas atrodo paprastesnis, todėl jį įgyvendinkime. Pirmiausia sukursime `TextVectorization` sluoksnį su nurodytu žodynu, paimtu iš Word2Vec įterpimų:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim žodžių įterpimų biblioteka turi patogią funkciją, `get_keras_embeddings`, kuri automatiškai sukurs atitinkamą Keras įterpimų sluoksnį jums.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Viena iš priežasčių, kodėl nematome didesnio tikslumo, yra ta, kad kai kurių žodžių iš mūsų duomenų rinkinio nėra iš anksto apmokyto GloVe žodyno, todėl jie iš esmės ignoruojami. Norėdami tai įveikti, galime apmokyti savo žodžių įterpimus, remdamiesi mūsų duomenų rinkiniu.


## Kontekstiniai įterpiniai

Viena pagrindinių tradicinių iš anksto apmokytų įterpinių, tokių kaip Word2Vec, apribojimų yra tai, kad, nors jie gali užfiksuoti tam tikrą žodžio reikšmę, jie negali atskirti skirtingų reikšmių. Tai gali sukelti problemų vėlesniuose modeliuose.

Pavyzdžiui, žodis „play“ turi skirtingas reikšmes šiuose dviejuose sakiniuose:
- Aš nuėjau į **spektaklį** teatre.
- Jonas nori **žaisti** su savo draugais.

Iš anksto apmokyti įterpiniai, apie kuriuos kalbėjome, abu žodžio „play“ reikšmes pateikia viename įterpinyje. Norint įveikti šį apribojimą, reikia kurti įterpinius, pagrįstus **kalbos modeliu**, kuris yra apmokytas naudojant didelį tekstų korpusą ir *žino*, kaip žodžiai gali būti naudojami skirtinguose kontekstuose. Kontekstinių įterpinių aptarimas nėra šio mokymo dalis, tačiau prie jų sugrįšime, kai kalbėsime apie kalbos modelius kitame skyriuje.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
